In [8]:
import os
from dotenv import load_dotenv
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
import base64
import email
import time
from datetime import datetime
import logging
from google.oauth2.credentials import Credentials
from google.auth.transport.requests import Request
import pickle
import atexit
from groq import Groq
import json
from langfuse import Langfuse
from email.message import EmailMessage

from langfuse.openai import openai

# Load environment variables from .env file
load_dotenv()

# Define the scopes you need
SCOPES = ['https://www.googleapis.com/auth/gmail.modify']


def get_gmail_service():
    creds = None
    # Token file to store credentials
    token_file = 'token.pickle'
    
    try:
        # Try to load existing credentials
        if os.path.exists(token_file):
            print("Loading existing credentials")
            with open(token_file, 'rb') as token:
                creds = pickle.load(token)
        
        # If no valid credentials available, authenticate
        if not creds or not creds.valid:
            if creds and creds.expired and creds.refresh_token:
                creds.refresh(Request())
            else:
                flow = InstalledAppFlow.from_client_secrets_file(
                    'credentials.json', 
                    SCOPES,
                    redirect_uri='http://localhost:8080'
                )
                creds = flow.run_local_server(
                    port=8080,
                    prompt='consent'
                )
            # Save the credentials for future use
            with open(token_file, 'wb') as token:
                pickle.dump(creds, token)
        
        # Build the Gmail service
        service = build('gmail', 'v1', credentials=creds)
        print("Gmail service initialized successfully")
        return service
    except Exception as e:
        print(f"Error initializing Gmail service: {str(e)}")
        raise
    
get_gmail_service()

Loading existing credentials
Gmail service initialized successfully


In [10]:
def create_draft(service, to, subject, body, from_email=None):
    try:
        # Create a message
        message = EmailMessage()
        message.set_content(body)
        message["To"] = to
        if from_email:
            message["From"] = from_email
        message["Subject"] = subject
        
        # Encode the message
        encoded_message = base64.urlsafe_b64encode(message.as_bytes()).decode()
        
        # Create the draft
        create_message = {"message": {"raw": encoded_message}}
        draft = service.users().drafts().create(userId="me", body=create_message).execute()
        
        print(f'Draft created successfully! Draft ID: {draft["id"]}')
        return draft
    
    except Exception as error:
        print(f"An error occurred: {error}")
        return None

# Example usage:
draft = create_draft(
    service=get_gmail_service(),
    to="recipient@example.com",
    subject="Test Draft Email",
    body="This is a test draft email created using the Gmail API."
)


Loading existing credentials
Gmail service initialized successfully
Draft created successfully! Draft ID: r-6533839442592082726


In [11]:
def send_email(service, to, subject, body, from_email=None):
    try:
        # Create a message
        message = EmailMessage()
        message.set_content(body)
        message["To"] = to
        if from_email:
            message["From"] = from_email
        message["Subject"] = subject
        
        # Encode the message
        encoded_message = base64.urlsafe_b64encode(message.as_bytes()).decode()
        
        # Send the message
        create_message = {"raw": encoded_message}
        sent_message = service.users().messages().send(userId="me", body=create_message).execute()
        
        print(f'Message sent successfully! Message ID: {sent_message["id"]}')
        return sent_message
    
    except Exception as error:
        print(f"An error occurred: {error}")
        return None

# Example usage:
sent_message = send_email(
    service=get_gmail_service(),
    to="ashray.gupta@gmail.com",
    subject="Test Email",
    body="This is a test email sent using the Gmail API."
)


Loading existing credentials
Gmail service initialized successfully
Message sent successfully! Message ID: 19577fc47ab8affb


In [23]:
def reply_to_email(service, message_id, to, subject, body, from_email=None):
    """
    Reply to an existing email using Gmail API, ensuring proper threading across email clients.
    
    Args:
        service: Authenticated Gmail API service instance
        message_id: ID of the message being replied to
        to: Email address of the recipient
        subject: Subject line for the email
        body: Body text of the email
        from_email: Optional sender email address
        
    Returns:
        The sent message if successful, None otherwise
    """
    try:
        # Get the original message to extract necessary headers
        original_message = service.users().messages().get(userId="me", id=message_id, format="metadata", 
                                                         metadataHeaders=["Subject", "References", "Message-ID", "Thread-Index"]).execute()
        thread_id = original_message['threadId']
        
        # Extract headers from original message
        headers = original_message['payload']['headers']
        original_subject = next((h['value'] for h in headers if h['name'] == 'Subject'), '')
        original_message_id = next((h['value'] for h in headers if h['name'] == 'Message-ID'), f"<{message_id}@mail.gmail.com>")
        references = next((h['value'] for h in headers if h['name'] == 'References'), '')
        
        # Create a message
        message = EmailMessage()
        message.set_content(body)
        message["To"] = to
        if from_email:
            message["From"] = from_email
            
        # Ensure we keep the exact same subject as the original for better Outlook threading
        message["Subject"] = original_subject
        
        # Set threading headers in specific order for Outlook
        message["Thread-Index"] = thread_id  # Add Thread-Index for Outlook
        message["In-Reply-To"] = original_message_id
        message["References"] = f"{references} {original_message_id}".strip()
        
        # Add these Outlook-specific headers
        message["Thread-Topic"] = original_subject.replace("Re: ", "").strip()
        message["Thread-Parent"] = original_message_id
        
        # Encode the message
        encoded_message = base64.urlsafe_b64encode(message.as_bytes()).decode()
        
        # Send the message in the same thread
        create_message = {
            "raw": encoded_message,
            "threadId": thread_id
        }
        
        sent_message = service.users().messages().send(userId="me", body=create_message).execute()
        
        print(f'Reply sent successfully! Message ID: {sent_message["id"]}')
        return sent_message
    
    except Exception as error:
        print(f"An error occurred: {error}")
        return None

# Example usage:
reply_message = reply_to_email(
    service=get_gmail_service(),
    message_id="195780a6ee0d5c61",  # ID of the message you're replying to
    to="marmik.hc.1904@gmail.com",
    subject="Re: Original Subject",
    body="Thank you for your email. I'm happy to help with your inquiry."
)

Loading existing credentials
Gmail service initialized successfully
Reply sent successfully! Message ID: 195780b3821e59f0


In [22]:
# Function to get the last 10 emails from Gmail
def get_recent_emails(service, max_results=10):
    """
    Retrieves the most recent emails from Gmail.
    
    Args:
        service: Authenticated Gmail API service instance.
        max_results: Maximum number of emails to retrieve (default: 10).
        
    Returns:
        A list of message objects containing email data.
    """
    try:
        # Call the Gmail API to fetch recent messages
        results = service.users().messages().list(
            userId="me", 
            maxResults=max_results, 
            labelIds=['INBOX']
        ).execute()
        
        messages = results.get('messages', [])
        
        if not messages:
            print("No messages found.")
            return []
        
        # Get detailed message data for each message
        email_data = []
        for message in messages:
            msg = service.users().messages().get(userId="me", id=message['id']).execute()
            email_data.append(msg)
            
        print(f"Successfully retrieved {len(email_data)} recent emails.")
        return email_data
    
    except Exception as error:
        print(f"An error occurred: {error}")
        return []

# Example usage:
recent_emails = get_recent_emails(
    service=get_gmail_service(),
    max_results=10
)

# Display snippets of the retrieved emails
if recent_emails:
    print("\nEmail snippets:")
    for i, email in enumerate(recent_emails, 1):
        print(f"{i}. {email}...")


Loading existing credentials
Gmail service initialized successfully
Successfully retrieved 10 recent emails.

Email snippets:
1. {'id': '195780a6ee0d5c61', 'threadId': '1957807fb951660b', 'labelIds': ['IMPORTANT', 'CATEGORY_PERSONAL', 'INBOX'], 'snippet': 'Thank you for your response. On Sat, 8 Mar 2025 at 15:14, &lt;ashray.gupta@gmail.com&gt; wrote: Thank you for your email. I&#39;m happy to help with your inquiry.', 'payload': {'partId': '', 'mimeType': 'multipart/alternative', 'filename': '', 'headers': [{'name': 'Delivered-To', 'value': 'ashray.gupta@gmail.com'}, {'name': 'Received', 'value': 'by 2002:a05:6358:7295:b0:1da:3844:c638 with SMTP id w21csp529575rwf;        Sat, 8 Mar 2025 15:15:04 -0800 (PST)'}, {'name': 'X-Received', 'value': 'by 2002:a05:6808:309f:b0:3f6:7b56:d10d with SMTP id 5614622812f47-3f697be9abamr4369524b6e.33.1741475704726;        Sat, 08 Mar 2025 15:15:04 -0800 (PST)'}, {'name': 'ARC-Seal', 'value': 'i=1; a=rsa-sha256; t=1741475704; cv=none;        d=google.c